In [1]:
from data_provider.data_factory import data_provider
from utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from torch.utils.data import Subset
from tqdm import tqdm
from models.PatchTST import PatchTST
from models.GPT4TS import GPT4TS
from models.DLinear import DLinear
from models.TEMPO import TEMPO
# from models.T5 import T54TS
from models.ETSformer import ETSformer


import numpy as np
import torch

In [2]:
import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

In [3]:
def get_init_config(config_path=None):
    config = OmegaConf.load(config_path)
    return config

In [4]:
from omegaconf import OmegaConf

# Define the configuration dictionary
config = {
    "datasets": {
        "ECL": {
            "root_path": "/u/dcao1/workspace/Generative-TS-train_all/csdi_data/",
            "data_path": "electricity_nips",
            "data_name": "ECL",
            "data": "ecl",
            "lradj": "type4",
            "features": "M",
            "target": "400001",
            "embed": "timeF",
            "freq": 0,
            "percent": 100
        }
    }
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)


In [5]:

# Save the configuration to a YAML file
with open("./configs/custom_dataset.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration saved to ./configs/custom_dataset.yml")

Configuration saved to ./configs/custom_dataset.yml


In [12]:
config = {
    "description": "TEMPO",
    "checkpoints": "checkpoints/lora_revin_5domain_checkpoints_1/",
    "task_name": "long_term_forecast",
    "prompt": 1,
    "num_nodes": 1,
    "seq_len": 168,
    "pred_len": 30,
    "label_len": 30,
    "decay_fac": 0.5,
    "learning_rate": 0.001,
    "batch_size": 256,
    "num_workers": 0,
    "train_epochs": 10,
    "lradj": "type3",
    "patience": 5,
    "gpt_layers": 6,
    "is_gpt": 1,
    "e_layers": 3,
    "d_model": 768,
    "n_heads": 4,
    "d_ff": 768,
    "dropout": 0.3,
    "enc_in": 7,
    "c_out": 1,
    "patch_size": 16,
    "kernel_size": 25,
    "loss_func": "mse",
    "pretrain": 1,
    "freeze": 1,
    "model": "TEMPO",
    "stride": 8,
    "max_len": -1,
    "hid_dim": 16,
    "tmax": 20,
    "itr": 3,
    "cos": 1,
    "equal": 1,
    "pool": False,
    "no_stl_loss": False,
    "stl_weight": 0.001,
    "config_path": "./configs/custom_dataset.yml",
    "datasets": "ETTm1,ETTh1,ETTm2,electricity,traffic,weather",
    "target_data": "ECL",
    "use_token": 0,
    "electri_multiplier": 1,
    "traffic_multiplier": 1,
    "embed": "timeF",
    "percent": 100,
    "model_id": 'prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0'
    #"model_id": 'prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll168_pl24_dm768_nh4_el3_gl6_df768_ebtimeF_itr0' #"prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
    # "model_id": "prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)

# Save the configuration to a YAML file
with open("configs/etth2_config.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration written to config.yml")

Configuration written to config.yml


In [13]:
config = get_init_config(cfg.config_path)

In [14]:
SEASONALITY_MAP = {
   "minutely": 1440,
   "10_minutes": 144,
   "half_hourly": 48,
   "hourly": 24,
   "daily": 7,
   "weekly": 1,
   "monthly": 12,
   "quarterly": 4,
   "yearly": 1
}

In [15]:
device = torch.device('cuda:0')

In [16]:
if cfg.model == 'PatchTST':
    model = PatchTST(cfg, device)
    model.to(device)
elif cfg.model == 'DLinear':
    model = DLinear(cfg, device)
    model.to(device)
elif cfg.model == 'TEMPO':
    model = TEMPO(cfg, device)
    model.to(device)
elif cfg.model == 'T5':
    model = T54TS(cfg, device)
    model.to(device)
elif 'ETSformer' in cfg.model:
    model = ETSformer(cfg, device)
    model.to(device)
else:
    model = GPT4TS(cfg, device)


params = model.parameters()
model_optim = torch.optim.Adam(params, lr=cfg.learning_rate)

early_stopping = EarlyStopping(patience=cfg.patience, verbose=True)
if cfg.loss_func == 'mse':
    criterion = nn.MSELoss()
elif cfg.loss_func == 'smape':
    class SMAPE(nn.Module):
        def __init__(self):
            super(SMAPE, self).__init__()
        def forward(self, pred, true):
            return torch.mean(200 * torch.abs(pred - true) / (torch.abs(pred) + torch.abs(true) + 1e-8))
    criterion = SMAPE()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=cfg.tmax, eta_min=1e-8)

trainable params: 308736 || all params: 82207488 || trainable%: 0.38


In [17]:
model_path = os.path.join(cfg.checkpoints, cfg.model_id)
best_model_path = model_path + '/checkpoint.pth'
print(best_model_path)
model.load_state_dict(torch.load(best_model_path,  map_location=torch.device(device)), strict=False)
print("------------------------------------")

checkpoints/lora_revin_5domain_checkpoints_1/prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0/checkpoint.pth
------------------------------------


In [18]:
cfg.data = config['datasets'][cfg.target_data].data
cfg.root_path = config['datasets'][cfg.target_data].root_path
cfg.data_path = config['datasets'][cfg.target_data].data_path
cfg.data_name = config['datasets'][cfg.target_data].data_name
cfg.features = config['datasets'][cfg.target_data].features
cfg.freq = config['datasets'][cfg.target_data].freq
cfg.target = config['datasets'][cfg.target_data].target
cfg.embed = config['datasets'][cfg.target_data].embed
cfg.percent = config['datasets'][cfg.target_data].percent
cfg.lradj = config['datasets'][cfg.target_data].lradj
if cfg.freq == 0:
    cfg.freq = 'h'
cfg.seq_len = 168 + 24
cfg.pred_len = 24 
test_data, test_loader = data_provider(cfg, 'test')

self.enc_in = 370
self.data_x = (7, 370)
test 7


In [19]:
test_data.mean_data

array([1.15276130e+02, 1.47397511e+02, 6.35067272e+01, 1.01070956e+02,
       7.58405982e+01, 3.31081975e+01, 4.57085020e+02, 4.93287882e+02,
       9.47804234e+00, 2.22973609e+03, 2.44243172e+02, 1.97515179e+03,
       2.23693582e+03, 1.13372034e+02, 4.64249180e+02, 5.83736119e+02,
       1.11329022e+02, 8.63760072e+01, 1.00248877e+03, 8.58561621e+01,
       5.99623908e+01, 8.76977168e+02, 1.86530945e+02, 9.07697424e+02,
       6.42006662e+01, 3.73959144e+02, 4.32386718e+01, 8.12590213e+02,
       3.35626322e+01, 4.04535862e+01, 2.00329203e+02, 4.11144110e+01,
       2.63150090e+01, 1.15248079e+03, 3.19602954e+02, 9.21947516e+02,
       4.87250686e+04, 1.76907901e+04, 9.47384413e+01, 7.52868815e+02,
       4.76208078e+02, 8.06064311e+02, 4.82057762e+02, 3.50051273e+02,
       9.59660644e+01, 2.62262610e+03, 1.45180629e+01, 9.09056629e+01,
       1.63716938e+02, 1.99858671e+02, 3.64408540e+02, 2.24329067e+02,
       1.10133322e+03, 2.18013187e+02, 1.57736250e+02, 7.35230875e+01,
      

In [20]:
# %%
import torch.distributions as dist
preds = []
trues = []
# mases = []

# Initialize accumulators for errors
total_mae = 0
total_mse = 0
n_samples = 0
itr = 0

cfg.seq_len = 168
cfg.pred_len = 24
model.eval()
# import pdb; pdb.set_trace()
with torch.no_grad():
    for i in range(len(test_data)):
        #enumerate(test_loader): #tqdm(enumerate(test_loader), total=len(test_loader)):
        data = test_data[i]
        seq_x, text_embedding, observed_mask = data[0].to(device), data[1].to(device), data[2].to(device)
        seq_x = seq_x.float().to(device)
        batch_x = seq_x[:, :cfg.seq_len].unsqueeze(-1) # 137, 168, 1
        batch_y = seq_x[:,cfg.seq_len:cfg.seq_len+cfg.pred_len]
        trues.append(batch_y.cpu().numpy())

                        
        outputs, _ = model(batch_x, 0, None, None, None) #+ model(seq_seasonal, ii) + model(seq_resid, ii)

        mu, sigma, nu = outputs[0], outputs[1], outputs[2]
        # Create the Student's t-distribution with the predicted parameters
        student_t = dist.StudentT(df=nu, loc=mu, scale=sigma)

        # Generate 30 samples for each prediction
        num_samples = 35
        probabilistic_forecasts = student_t.rsample((num_samples,))

        # The shape of probabilistic_forecasts will be (num_samples, batch_size, pred_length)
        print(probabilistic_forecasts.shape)
        preds.append(probabilistic_forecasts.cpu().numpy())


torch.Size([35, 370, 30])
torch.Size([35, 370, 30])
torch.Size([35, 370, 30])
torch.Size([35, 370, 30])
torch.Size([35, 370, 30])
torch.Size([35, 370, 30])
torch.Size([35, 370, 30])


In [21]:
trues[0]

array([[-1.284714  , -1.4160967 , -1.4059902 , ..., -0.7692896 ,
        -0.99162954, -1.1331185 ],
       [-0.9577074 , -0.79033315, -0.77837783, ...,  0.17804651,
        -0.20452322, -0.7186013 ],
       [-1.0019546 , -1.1834075 , -1.1834075 , ...,  0.28117752,
        -0.5936852 , -0.81402105],
       ...,
       [-1.0161966 , -1.057863  , -1.1741474 , ..., -0.2270235 ,
        -0.99959636, -0.77541083],
       [-1.3774434 , -1.3334916 , -1.3422819 , ...,  0.04158786,
        -1.2411046 , -1.2982861 ],
       [-1.4698776 , -1.5704279 , -1.2486668 , ...,  0.27969885,
        -0.67888135, -0.8464653 ]], dtype=float32)

In [22]:
preds[0]

array([[[-1.4942299 ,  0.3466649 , -0.1794734 , ...,  1.5418237 ,
         -0.8713156 , -1.9530109 ],
        [-0.9860479 , -0.77252203, -0.638939  , ..., -1.271782  ,
         -1.1006117 , -0.6287196 ],
        [-1.1362414 , -1.8481455 , -0.8334619 , ..., -1.0091337 ,
          0.6884369 ,  0.79988897],
        ...,
        [-0.6154065 , -0.15223598, -1.4339099 , ..., -1.235589  ,
         -0.6562629 , -0.03243496],
        [-1.9273536 , -2.5457954 , -1.4822483 , ..., -1.9193256 ,
         -0.13780165, -0.3858933 ],
        [-1.3667933 , -0.0565958 , -1.4252726 , ..., -0.8693197 ,
         -2.3959048 , -0.6017202 ]],

       [[-1.7063351 , -1.6945823 ,  0.1893189 , ..., -1.2913091 ,
          0.82824945, -0.66409177],
        [-0.9802842 , -0.96147496,  0.09971511, ..., -1.4645338 ,
         -1.381603  , -1.4522662 ],
        [-1.4166349 , -1.0720152 , -0.6512529 , ..., -1.1616634 ,
          0.46041048,  0.48068497],
        ...,
        [-1.0131671 , -1.1718489 , -0.6194771 , ..., -

In [23]:
np.array(preds).shape, np.array(trues).shape

((7, 35, 370, 30), (7, 370, 24))

In [25]:
gt_data = np.array(trues)
synthetic_data = np.array(preds)

In [26]:
synthetic_data = synthetic_data[:,:,:, :gt_data.shape[2]]
target_mask = np.ones_like(gt_data)

In [135]:
# synthetic_data = np.swapaxes(synthetic_data, -1, -2)
# gt_data = np.swapaxes(gt_data, -1, -2)
# synthetic_data.shape

In [27]:
from metrics.imputation_metrics import mse_withmask, mae_withmask, calc_quantile_CRPS, calc_quantile_CRPS_sum

In [28]:
calc_quantile_CRPS_sum(torch.Tensor(gt_data),torch.Tensor(synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

0.3561742933172929

In [138]:
# synthetic_data = np.array(synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
# gt_data = np.array(gt_data).reshape(gt_data.shape[0],gt_data.shape[2], -1)
# target_mask = np.ones_like(gt_data)

In [29]:
unormzalized_gt_data = []
for g in gt_data:
    unormzalized_gt_data.append(np.transpose(np.transpose(g)*test_data.std_data+test_data.mean_data))
unormzalized_gt_data = np.array(unormzalized_gt_data)
unormalized_synthetic_data = []
for i in range(len(synthetic_data)):
    for j in range(len(synthetic_data[i])):
        s = synthetic_data[i][j]
        unormalized_synthetic_data.append(np.transpose(np.transpose(s)*test_data.std_data+test_data.mean_data))
unormalized_synthetic_data = np.array(unormalized_synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
unormzalized_gt_data.shape,unormalized_synthetic_data.shape

((7, 370, 24), (7, 35, 370, 24))

In [140]:
unormzalized_gt_data.shape

(7, 370, 24)

In [141]:
unormalized_synthetic_data.shape

(7, 35, 370, 24)

In [142]:
gt_data.shape

(7, 370, 24)

In [30]:
unormalized_synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)
unormzalized_gt_data = np.swapaxes(unormzalized_gt_data, -1, -2)


In [31]:
target_mask = np.ones_like(unormzalized_gt_data)

In [32]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.08128854475523296

In [33]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.08128854475523296

In [34]:
print("all:   CRPS_sum:")
print(calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1))
print("all: CRPS_sum:")
# import pdb; pdb.set_trace()
CRPS_sum_1 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,:10]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_2 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,10:20]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_3 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,20:30]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

CRPS_sum_mean = np.mean([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
CRPS_sum_std = np.std([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
print("Mean CRPS_sum:", CRPS_sum_mean)
print("Standard Deviation CRPS_sum:", CRPS_sum_std)

all:   CRPS_sum:
0.08128854475523296
all: CRPS_sum:
Mean CRPS_sum: 0.08956774075826009
Standard Deviation CRPS_sum: 0.002802724887745557


In [36]:
from metrics.WIS import score

In [37]:
synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)

labels = np.swapaxes(unormzalized_gt_data, -1, -2)

In [38]:
low_q = np.quantile(synthetic_data,0.05,axis=1)
high_q = np.quantile(synthetic_data,0.95,axis=1)
mid_q = np.quantile(synthetic_data,0.5,axis=1)

In [39]:
high_q_reshape = np.reshape(high_q,newshape=(-1,high_q.shape[-1]))
low_q_reshape = np.reshape(low_q,newshape=(-1,low_q.shape[-1]))
gt_data_reshape = np.reshape(labels,newshape=(-1,labels.shape[-1]))
print(high_q_reshape.shape,low_q_reshape.shape,gt_data_reshape.shape)

(2590, 24) (2590, 24) (2590, 24)


In [40]:
alpha = 0.05 # to be set by competition hosts
if not(alpha > 0 and alpha <= 1):
    raise ParticipantVisibleError(f'alpha should be (0,1]. Found {alpha}')

MWIS,coverage = score(gt_data_reshape,high_q_reshape,low_q_reshape,alpha)

if not(coverage >= (1-alpha)):
    raise ParticipantVisibleError(f'Coverage was below {(1-alpha)*100}%')

In [41]:
MWIS,coverage

(1359.1446534926044, 22.544015444015443)